In [1]:
print('Lecture-6')

Lecture-6


### Veri Kümesindeki Eksik ve Geçersiz Verilerin Ele alınması (Lecture-27 devamı)

Veri kümesini eksik veri bakımından analiz ettikten sonra şimdi de eksik veirlerin nasıl ele alınacağı üzerinde duralım. 

Melbourne Housing Snapshot: https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot?resource=download

melbourne'deki gayrimenkul analizi için hazırlanmış bir veri kümesidir. MHS. 21 özellikten oluşmaktadır.

In [2]:
import pandas as pd

df = pd.read_csv('melb_data.csv')

missing_cols = [col_name for col_name in df.columns if df[col_name].isna().any()]
total_missing_rows = pd.isna(df).any(axis=1).sum()
total_missing_rows_ratio = total_missing_rows / len(df)

print(f'Toplam Sütun Sayısı: {df.shape[1]}')
print(f'Toplam Satır Sayısı: {df.shape[0]}')
print(f'Eksik verilerin bulunduğu sütunlar: {missing_cols}')
print(f'Eksik verilerin bulunduğu satırların sayısı: {total_missing_rows}')
print(f'Eksik verilerin bulunduğu satırların Oranı: {total_missing_rows_ratio}')

Toplam Sütun Sayısı: 21
Toplam Satır Sayısı: 13580
Eksik verilerin bulunduğu sütunlar: ['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']
Eksik verilerin bulunduğu satırların sayısı: 7384
Eksik verilerin bulunduğu satırların Oranı: 0.543740795287187


Eksik verilerin ele alınmasında temel olarak iki yöntem kullanılmaktadır.

1) Eksik verilerin bulunduğu satır ya da sütunların tamamen atılması yöntemi.
2) Eksik verilerin başka değerlerle doldurulması (imputation) yöntemi.

Eksik veriler özellikle belli sütunlarda oluşmuşsa eksik verilerin bulunduğu satırlaın atılması "yanlı (biased)" bir veri kümesinin oluşmasına yol açabilecektir. 

dropna.

In [3]:
deleted_rows_df = df.dropna(axis=0)
print(deleted_rows_df.shape)

(6196, 21)


In [4]:
deleted_cols_df = df.dropna(axis=1)
print(deleted_cols_df.shape)

(13580, 17)


Eksik verilerin ele alınmasında diğer bir strateji de eksik veriler yerine başka değerlerin yerleştirilmesi işlemidir. Buna "imputation" denilmektedir. 

- Eksik verilerin ilgili sütun ortalaması, medyanı ya da mod değeri ile doldurulması yöntemi -> en sık kullanılan yöntem
- Eksik verilerin sütundaki önceki verilerle ya da sonraki verilerle doldurulması yöntemi
- Eksik değerlerin tahmin yapma yoluyla doldurulması yöntemi
- Eksik verilerin kategorik sütunlarda farklı bir sınıf ile temsil edilmesi yöntemi
- Eksik verilerin KNN (k-Nearest Neighbours) algoritmasıyla doldurulması yöntemi
- Eksik verilerin rastgele değerlerle doldurulması yöntemi

MHS --> ['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']
'CAR' -> Araba sayısına ilişkin bir değer. ayrık oransal bir ölçek. Bu durumda eksik verileri sütun ortalamasına göre doldurabiliriz. Ancak ortalama aldığımızda sayılar ayrık olmaktan çıkabilecektir. Eğer değerlerin ayrıklığı korunmak isteniyorsa yuvarlama yapılması uygun olabilmektedir. 

In [7]:
import numpy as np

print('Car sütununun ortalaması: {}'.format(df['Car'].mean()))
df['Car'].fillna(np.round(df['Car'].mean()), inplace=True)

Car sütununun ortalaması: 1.6118556701030928


In [9]:
df['Car'].isna().sum()

0

Veri kümemizdeki "BuildingArea" sütunu da "ayrık sayısal (discrete numeric) oransal ölçeğe" ilişkin bir sütundur.


In [10]:
print('BuildingArea sütununun ortalaması: {}'.format(df['BuildingArea'].mean()))
df['BuildingArea'].fillna(np.round(df['BuildingArea'].mean()), inplace=True)

BuildingArea sütununun ortalaması: 151.96764988779805


In [11]:
df['BuildingArea'].isna().sum()

0

In [12]:
df['BuildingArea']

0        152.0
1         79.0
2        150.0
3        152.0
4        142.0
         ...  
13575    152.0
13576    133.0
13577    152.0
13578    157.0
13579    112.0
Name: BuildingArea, Length: 13580, dtype: float64

"YearBuilt" sütunu binanın yapım yılını belirtmektedir. Biz burada bu yıl bilgisinin sıralı bir ölçeğer ilişkin olduğunu kabul edeceğiz ve doldurma işleminde median kullanacağız.

In [14]:
print('YearBuilt sütununun Medianı: {}'.format(df['YearBuilt'].median()))

YearBuilt sütununun Medianı: 1970.0


In [17]:
df['YearBuilt'].min()

1196.0

In [18]:
df['YearBuilt'].max()

2018.0

In [21]:
df['YearBuilt'].fillna(df['YearBuilt'].median(), inplace=True)

In [22]:
df['YearBuilt']

0        1970.0
1        1900.0
2        1900.0
3        1970.0
4        2014.0
          ...  
13575    1981.0
13576    1995.0
13577    1997.0
13578    1920.0
13579    1920.0
Name: YearBuilt, Length: 13580, dtype: float64

CouncilArea kategorik verilerden oluşmaktadır. Bu nedenle sütundaki eksik verileri mode işlemi ile dolduracağız.

In [23]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,152.0,1970.0,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,152.0,1970.0,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [30]:
print(type(df['CouncilArea'].mode()))
print('CouncilArea sütununun Mode: {}'.format(df['CouncilArea'].mode()[0]))

<class 'pandas.core.series.Series'>
CouncilArea sütununun Mode: Moreland


In [31]:
df['CouncilArea'].fillna(df['CouncilArea'].mode()[0], inplace=True)

In [32]:
df['CouncilArea']

0           Yarra
1           Yarra
2           Yarra
3           Yarra
4           Yarra
           ...   
13575    Moreland
13576    Moreland
13577    Moreland
13578    Moreland
13579    Moreland
Name: CouncilArea, Length: 13580, dtype: object

In [33]:
df.isna().sum().sum()

0

## Scikit-Learn Kütüphanesinin Kullanımı

Makine öğrenmesi uygulamaları için hazırlanmış olan açık kaynak kodlu bir kütüphanedir. 
pip install sklearn

Scikit-Learn kütüphanesi büyük ölçüde nesne yönelimli bir kullanım için hazırlanmıştır. Pek çok sınıf vardır. Sınıf ismi C olmak üzere bu işlemler tipik olarka şöyle yürütülür:

1) Önce C sınıfı türünden bir nesne yaratılır. Nesne yaratılırken sınıfın `__init__` metodunda bazı parametreler için değerler girilebilmektedir. Scikit-learn kütüphanesindeki fonksiyonların ve metotların parametreleri kolay bir kullanım sağlamak için default argümanlar almıştır.

c = C()

2) İlgili sınıf türünden nesne yarattıktan sonra bu nesne ile ilgili sınıfın `fit` metodu çağrılır. fit metodu aldığı veriler üzerinde işlemler yaparak birtakım bilgiler elde eder ve bu bilgileri nesnenin bazı örnek özniteliklerinde saklar. fit işlemi bir dönüştürme ya da kestirim yapmamaktadır. yalnızca aldığı veriler üzerinde dönüştürme ya da kestirim için gerekli olan bilgileri elde edip saklamaktadır. bu anlamda fit işlemi bir eğitim işlemi olarak ele alınabilir.

c.fit(fit_data)

3) fit işleminden sonra asıl dönüştürme ya da kestirim işlemini yapacak olan transform metodu çağrılır. 

transformed_data = c.transform(data)

transform metodu fit metodu ile elde edilen bilgiler üzerinde dönüştürme ya da kestirim yaparak dönüştürülmüş veri kümesini geri dönüş değeri olarak vermektedir. Eğitim fit ile yapıldıktan sonra transform metodu birden fazla kez çağrılarak farklı veriler dönüştürülebilir.

transformed_data1 = c.transform(data1)
transformed_data2 = c.transform(data2)
transformed_data3 = c.transform(data3)

4) Bazen fit işleminde kullanılacak veri ile dönüştürülecek veri aynı olabilir.

c.fit(data)
transformed_data = c.transform(data)

Böylesi durumlarda bu işlemin tek hamlede yapılabilmesi için ilgili sınıflarda fit_transform metotları bulundurulmuştur.

transformed_data = c.fit_transform(data)

= 

c.fit(data)
transformed_data = c.transform(data)

https://scikit-learn.org/stable/



## Kategorik (Nominal) ve Sıralı (Ordinal) Verilerin Sayısal Biçime Dönüştürülmesi

Devam edilecek.

Ödev: https://www.kaggle.com/datasets/zafarali27/car-price-prediction/data